In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def get_trading_dates(date_str, num_days=4):
    """
    Get the preceding dates and the given date. If the given date is not a Friday,
    find the most recent Friday before that date.
    
    Parameters:
    -----------
    date_str : str
        Date in format 'YYYY-MM-DD' (e.g., '2025-07-04')
    num_days : int, default=4
        Number of preceding dates to return
    
    Returns:
    --------
    list
        List of date strings in 'YYYY-MM-DD' format (4 preceding dates + Friday)
    """
    # Parse the input date
    given_date = pd.to_datetime(date_str)
    
    # Find the most recent Friday
    if given_date.weekday() == 4:  # Already a Friday
        friday_date = given_date
        print(f"Given date {date_str} is already a Friday")
    else:
        # Go back in time to find the most recent Friday
        days_to_subtract = (given_date.weekday() + 3) % 7  # Calculate days back to Friday
        friday_date = given_date - pd.DateOffset(days=days_to_subtract)
        day_name = given_date.strftime('%A')
        print(f"Given date {date_str} is a {day_name}, using most recent Friday: {friday_date.strftime('%Y-%m-%d')}")
    
    # Generate the preceding dates from the Friday
    all_dates = []
    for i in range(num_days, 0, -1):  # Count backwards from num_days to 1
        prev_date = friday_date - pd.DateOffset(days=i)
        all_dates.append(prev_date.strftime('%Y-%m-%d'))
    
    # Add the Friday date
    all_dates.append(friday_date.strftime('%Y-%m-%d'))
    
    return all_dates

# # Example usage:
# try:
#     date_input = '2025-07-04'  # This is a Friday
#     dates = get_preceding_dates_and_friday(date_input)
#     print(f"Input date: {date_input}")
#     print(f"4 preceding dates + Friday: {dates}")
#     # Output: ['2025-06-30', '2025-07-01', '2025-07-02', '2025-07-03', '2025-07-04']
# except Exception as e:
#     print(f"Error: {e}")

# # Test with a non-Friday date:
# try:
#     date_input = '2025-07-05'  # This is a Saturday
#     dates = get_preceding_dates_and_friday(date_input)
#     print(f"Input date: {date_input}")
#     print(f"4 preceding dates + Friday: {dates}")
#     # Should find Friday 2025-07-04 and return the week ending on that Friday
# except Exception as e:
#     print(f"Error: {e}")

# # Test with a Monday:
# try:
#     date_input = '2025-07-07'  # This is a Monday
#     dates = get_preceding_dates_and_friday(date_input)
#     print(f"Input date: {date_input}")
#     print(f"4 preceding dates + Friday: {dates}")
#     # Should find Friday 2025-07-04 and return the week ending on that Friday
# except Exception as e:
#     print(f"Error: {e}")

In [ ]:
from ib_async import *
import pandas as pd
import json

util.startLoop()
ib = IB()
ib.connect(port=4001, clientId=0)
# gdx = Contract(symbol='GDX', exchange='SMART', secType='STK', currency='USD')
# nugt = Contract(symbol='NUGT', exchange='SMART', secType='STK', currency='USD')
# dust = Contract(symbol='DUST', exchange='SMART', secType='STK', currency='USD')
# bitu = Contract(symbol='BITU', exchange='SMART', secType='STK', currency='USD')
# sbit = Contract(symbol='SBIT', exchange='SMART', secType='STK', currency='USD')
# gld = Contract(symbol='GLD', exchange='SMART', secType='STK', currency='USD')
# ivv = Contract(symbol='IVV', exchange='SMART', secType='STK', currency='USD')
# uso = Contract(symbol='USO', exchange='SMART', secType='STK', currency='USD')


Error 162, reqId 85: Historical Market Data Service error message:API historical data query cancelled: 85, contract: Contract(secType='STK', symbol='SBIT', exchange='SMART', currency='USD')


In [ ]:
def get_contract(symbol):
    contract = Contract(symbol=symbol.upper(), exchange='SMART', secType='STK', currency='USD')
    return {"symbol": symbol.upper(), "contract": contract}


In [ ]:
symbols = ['gdx', 'nugt', 'dust', 'bitu', 'sbit', 'gld', 'ivv', 'uso']
symbols = ['gdx', 'nugt', 'dust', 'bitu', 'sbit']
contracts = [get_contract(symbol) for symbol in symbols]
contracts

In [ ]:
from datetime import datetime
date = datetime.now().strftime('%Y-%m-%d')
print(date)

# date = '2025-06-30'

dates = get_trading_dates(date)
dates

In [11]:
for contract in contracts:
    all_data = []  # Store all days' data for this contract
    
    for day in dates:
        print(f"Processing {contract['symbol']} on {day}")
        end_date = pd.to_datetime(f'{day} 00:00:00-04:00')
        # Add 1 day to the end_date
        end_date += pd.DateOffset(days=1)
        end_date = end_date.tz_convert('US/Eastern')

        bars = ib.reqHistoricalData(
            contract=contract['contract'], 
            endDateTime=end_date, 
            durationStr='1 D', 
            barSizeSetting='5 secs', 
            whatToShow='TRADES', 
            useRTH=False, 
            keepUpToDate=False
        )
        
        # Add each bar to the all_data list
        for b in bars:
            all_data.append([
                str(b.date)[:19], 
                b.open, 
                b.high, 
                b.low, 
                b.close, 
                b.volume, 
                b.barCount, 
                b.average
            ])
    
    # Create DataFrame with all days' data for this contract
    df = pd.DataFrame(
        all_data,
        columns=[
            "date",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "barCount",
            "average",
        ],
    )
    # Remove duplicate dates (keep the last occurrence)
    original_rows = len(df)
    df = df.drop_duplicates(subset=['date'], keep='last')
    deduplicated_rows = len(df)
    
    if original_rows != deduplicated_rows:
        print(f"Removed {original_rows - deduplicated_rows} duplicate date entries for {contract['symbol']}")
    
    # Save one CSV per contract with all days' data
    filename = f"{contract['symbol']}_{dates[0]}_to_{dates[-1]}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} rows to {filename}")

Processing SBIT on 2025-06-23


reqHistoricalData: Timeout for Contract(secType='STK', symbol='SBIT', exchange='SMART', currency='USD')


Processing SBIT on 2025-06-24
Processing SBIT on 2025-06-25
Processing SBIT on 2025-06-26
Processing SBIT on 2025-06-27
Saved 45653 rows to SBIT_2025-06-23_to_2025-06-27.csv
Processing GLD on 2025-06-23
Processing GLD on 2025-06-24
Processing GLD on 2025-06-25
Processing GLD on 2025-06-26
Processing GLD on 2025-06-27
Saved 57600 rows to GLD_2025-06-23_to_2025-06-27.csv
Processing IVV on 2025-06-23
Processing IVV on 2025-06-24
Processing IVV on 2025-06-25
Processing IVV on 2025-06-26
Processing IVV on 2025-06-27
Saved 57313 rows to IVV_2025-06-23_to_2025-06-27.csv
Processing USO on 2025-06-23
Processing USO on 2025-06-24
Processing USO on 2025-06-25
Processing USO on 2025-06-26
Processing USO on 2025-06-27
Saved 57574 rows to USO_2025-06-23_to_2025-06-27.csv
